In [ ]:
import numpy as np
import pandas as pd
import fastf1
from collections import defaultdict, Counter

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
# Dictionary to hold strategy -> total points
strategy_points = defaultdict(int)

# Counter for compound usage across all races
compound_counter = Counter()

# Counter for strategy types (1-stop, 2-stop, etc.)
strategy_type_counter = Counter()

In [ ]:
# Loop over all completed races
for rnd in range(1,15):  # adjust if more races are done
    session = fastf1.get_session(2025, rnd, 'R')
    session.load(laps=True, telemetry=False, weather=False)

    # Results table with driver points + status
    results = session.results.set_index('Abbreviation')

    # Go through all drivers
    for drv in session.drivers:
        drv_abbr = session.get_driver(drv)['Abbreviation']
        if rnd==10:
            if drv == '14':
                drv_laps = session.laps.pick_drivers(drv_abbr).iloc[:-4]
            else:
                drv_laps = session.laps.pick_drivers(drv_abbr).iloc[:-3]
        else:
            drv_laps = session.laps.pick_drivers(drv_abbr)
            if rnd==6:
                if drv_laps['Compound'].nunique() == 3:
                    drv_laps['Compound'] = drv_laps['Compound'].apply(lambda x: np.nan if x == 'nan' else x)
                    drv_laps['Stint'].bfill(inplace=True)
                    drv_laps['Compound'].bfill(inplace=True)
                else:
                    drv_laps['Stint'].fillna(0)
                    drv_laps['Compound'] = drv_laps['Compound'].apply(lambda x: 'MEDIUM' if x == 'nan' else x)
            if rnd == 13:
                drv_laps['Compound'] = drv_laps['Compound'].apply(lambda x: 'MEDIUM' if x == 'None' else x)

        if drv_laps.empty:
            continue

        # Skip drivers who retired / did not finish
        if drv_abbr in results.index:
            status = results.loc[drv_abbr, 'Status']
            if isinstance(status, str) and ('Retired' in status or 'DNF' in status or 'Did not start' in status or 'DNS' in status):
                continue
        else:
            continue

        # Count compounds (for global pie chart)
        compound_counter.update(drv_laps['Compound'])

        # Get stint info -> strategy string (SMH etc.)
        stints = drv_laps[['Stint', 'Compound']].drop_duplicates(subset=['Stint'])
        strategy_str = ''.join(stints['Compound'].str[0])  # e.g. SOFT->S

        # Add to strategy type counter (#stops = len(stints)-1)
        strategy_type = f"{len(stints)-1}-stop"
        strategy_type_counter[strategy_type] += 1

        # Assign points from results table
        points = results.loc[drv_abbr, 'Points']
        strategy_points[strategy_str] += points

In [ ]:
# ---- Final outputs ----

# Strategy points table
strategy_points_table = pd.DataFrame.from_dict(strategy_points, orient='index', columns=['Points'])
strategy_points_table = strategy_points_table.sort_values(by='Points', ascending=False)

# Compound usage counts
compound_usage = pd.Series(compound_counter).sort_values(ascending=False)

# Strategy type counts
strategy_type_usage = pd.Series(strategy_type_counter).sort_index()

In [ ]:
strategy_points_table

In [ ]:
compound_usage/compound_usage.sum()

In [ ]:
strategy_type_usage